# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import pandas as pd 
credit_score_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Yandex.Prakticum/2. Исследование надёжности заёмщиков/data.csv') #прочитаем данные и поместим в переменную credit_score_data

credit_score_data.head(21525)                   #выведем таблицу на экран           

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [45]:
credit_score_data.info()   #получим общее понимание набора данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [46]:
credit_score_data.describe() #Чтобы узнать разброс значений, средние величины, а также квантили, используем метод .describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Данные состоят из строк в количестве 21 525, 12 стобцов и всего 3 типов данных.
В столбцах days_employed и total_income есть пропущенные значения  - пропусков порядка 10% от всех строк. 
Пустые значения столбца total_income явно связанны с отсутствием значений в столбце days_employed. Возможно, это отсутствие опыта, или опыт есть, но он официально нигде не зафиксирован, человек работал без трудовой и не получал официальный доход, возможно имел бизнес, доход по которому никак не декларировался, т.е. скорее всего отсутствие значений обусловлено человеческим фактором.
 
В столбцах children наблюдаем отрицательное количество детей, аналогично и в столбце days_employed, чего быть не может. 
В дальнейшем мы исправим это.

Помимо этого, можно наблюдать аномальные значения в виде четырехсот тысяч дней в столбце days_employed и 20-ти детей в столбце children, в дальнейшем они могут испортить нам статистику. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [47]:
credit_score_data[credit_score_data['days_employed'].isna()].head() #выведем на экран пропущенные значения в столбце days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Заменим пропущенные значения в стобце days_employed на нули

In [48]:
credit_score_data['days_employed'].fillna(0, inplace=True) #заменим значения NaN на 0

In [49]:
#приведем значения в столбце с указанием образования к нижнему регистру, чтобы группировка была корректной
credit_score_data['education'] = credit_score_data['education'].str.lower()

В столбце total_income пропущенные значения заменим на медиану, т.к. разброс значений по данной колонке составляет от 167 тысяч рублей до 2 млн руб., не уверен что среднее использовать корректно

In [50]:
# группируем датафрейм по столбцам образования и типу дохода, и заполняем пропуски по группам ср.знач. по группе
credit_score_data.loc[:, 'total_income'] = credit_score_data.groupby(['education', 'income_type' ]).transform(lambda x: x.fillna(x.median()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  


Проверим,что нулевые значения убраны

In [51]:
credit_score_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [52]:
credit_score_data.isnull().sum() #проверим кол-во пропусков по каждому столбцу

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверим пропуски в столбцах с типом данных str

In [53]:
for column in credit_score_data.select_dtypes(include=['object']).columns.tolist(): #выбираем из столбцов тип данных object
    print(credit_score_data[column].value_counts())
    print('Итого:',credit_score_data[column].value_counts().sum(), end='\n\n') #строка кода, чтобы выводился тотал 
    
    
 

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
Итого: 21525

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
Итого: 21525

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
Итого: 21525

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64
Итого: 21525

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операц

**Вывод**

1. Мы убрали пропуски в столбцах days_employed и total_income. Пропущенные значения содержались в стобцах с количество дней общего трудового стажа и общимдоходом.
Мы заменили пропущенные значения в столбце total_income на медианные значения по группам образования и типу занятости, чтобы в целом статистика не была искажена.
2. Пропуски по столбцам наблюдались в одних и техже строках. Пропуски в данных столбцах имеют, вероятнее всего, человеческий фактор происхождения: например, кто-то на пенсии, и не работают, поэтому пропускают данное поле для заполнения, в связи с этим и общий доход также отсутствует. В целом, пропуски составляли 10% среди всех строк в наборе данных.

### Замена типа данных

Изучим общую информацию о типах данных нашей базы

In [54]:
credit_score_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбце days_employed должны находится целочисленные значения, отражающие количество дней стажа. Необходимо перобразовать в int.

Также, округлим значения дохода до целочисленного.

In [55]:
credit_score_data['days_employed'] = credit_score_data['days_employed'].astype('int')
credit_score_data['total_income'] = credit_score_data['total_income'].astype('int')

Исправим отрицательные значения из столбцов children и days_employed на положительные

In [56]:
credit_score_data['children'] = credit_score_data['children'].apply(lambda x: abs(x))
credit_score_data['days_employed'] = credit_score_data['days_employed'].apply(lambda x: abs(x))
credit_score_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.543275,60155.970128,43.293380,0.817236,0.972544,0.080883,1.653532e+05
std,1.379876,133355.906156,12.574584,0.548138,1.420324,0.272661,9.814856e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,610.000000,33.000000,1.000000,0.000000,0.000000,1.077190e+05
50%,0.000000,1808.000000,42.000000,1.000000,0.000000,0.000000,1.434960e+05
75%,1.000000,4779.000000,53.000000,1.000000,1.000000,0.000000,1.981490e+05
max,20.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Мы видим, что в наших данных присутствуют артефакты: 20 детей в столбце children, а также 400 тысяч дней в столбце days_employed: очевидно, что человек не может работать тысячу лет, и это сильно искажает наши данные.

In [57]:
credit_score_data = credit_score_data.drop(credit_score_data[credit_score_data.children == 20].index) #удалим строку, где значение детей равно 20
credit_score_data = credit_score_data.drop(credit_score_data[credit_score_data.days_employed > 100000].index) #удалим все строки, где кол-во дней больше 100 тысяч дней

credit_score_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,18011.000000,18011.000000,18011.000000,18011.000000,18011.000000,18011.000000,1.801100e+04
mean,0.547332,2070.422797,40.277775,0.798568,0.970851,0.086114,1.707315e+05
std,0.786561,2292.744105,10.974817,0.552276,1.439502,0.280540,1.003829e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.136700e+04
25%,0.000000,450.000000,32.000000,0.000000,0.000000,0.000000,1.134605e+05
50%,0.000000,1349.000000,40.000000,1.000000,0.000000,0.000000,1.481090e+05
75%,1.000000,2896.000000,48.000000,1.000000,1.000000,0.000000,2.017850e+05
max,5.000000,18388.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Стобец days_employed должен иметь типа данных как целочесленный, т.к.это количество дней, а также мы исправили тип данных по столбцу total_income на целочисленный для удобства. Помимо этого исправили отрицательные значения по стобцу с  количеством детей. Также мы видим, что теперь макисмлаьное количество рабочих дней соотносится с реальностью.

### Обработка дубликатов

Узнаем общее количество дубликатов строк

In [58]:
credit_score_data.duplicated().sum() #получим суммарное кол-во дубликатов

71

In [59]:
credit_score_data['education'] = credit_score_data['education'].str.lower()
print(credit_score_data['education'].value_counts())

среднее                12386
высшее                  4725
неоконченное высшее      707
начальное                189
ученая степень             4
Name: education, dtype: int64


In [60]:
credit_score_data.drop_duplicates(keep='first', inplace=True) #удалим дубликаты
credit_score_data.duplicated().sum()

0

**Вывод**

Мы удалили дубликаты строк, их было 71. В нашем случае в качестве дубликата нельзя выделить какой-либо один элемент, хотя таким элементом мог бы быть ID конкретного человека, имя и фамилия, который хочет взять кредит. В данном случае данные не персонализированы, поэтому мы искали полные дубликаты. 

Причины появления дубликатов могу быть как технические, так и иметь человеческий фактор: люди оставляли заявки на получение кредита, и получив отрицательный результат - вносили данные снова. 

### Лемматизация

Первым делом импортируем бибилиотеку nltk иpymystem3

In [61]:
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian') 
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

Найдем и выведем частоту каждой цели (элемента), встречающуюся в столбце purpose при помощи цикла

In [ ]:
list_of_purpose = credit_score_data['purpose'].tolist()

lemmas = []
for item in  list_of_purpose: 
  text = ' '.join(m.lemmatize(item))

lemmas.append(text)

Counter(lemmas)

In [ ]:
credit_score_data['purpose_lemm'] = credit_score_data['purpose'].apply(m.lemmatize)
credit_score_data['purpose_lemm']

Далее, мы напишем функцию, которая позволит нам записать в новый столбец укрупненную группу, если подоходящее слово есть в словаре. Так мы получим столбец с общими категориями целей.

In [ ]:
# определяем из списка лемм целевую группу и записываем ее в словарь general_purpose_groups

general_purpose_groups = {'недвижимость' : 'Недвижимость',
               'жилье' : 'Недвижимость', 
               'образование' : 'Образование', 
               'свадьба' : 'Проведение свадьбы', 
               'автомобиль' : 'Приобретение автомобиля'
              }
# создадим функцию, возвращающую значение по ключю из основной группы, если слово из листа лемм есть в словаре

def func(purp_item_list):
    for word in purp_item_list:
        
        if word in general_purpose_groups:
            return general_purpose_groups[word]

credit_score_data['new_purp_category'] = credit_score_data['purpose_lemm'].apply(func) # добавляем в новый столбец категории по целям кредитования 
credit_score_data.head()

**Вывод**

Мы выделили леммы  из всей базы целей, выделили оттуда общие категории и добавили новый столбец.  
Мы выделили следующие категории: Покупка или строительство недвижимости, свадьба, покупка автомобился и образование. Далее мы будем работать именно с этими категориями.

### Категоризация данных

Для дальнейшего анализа разделим уровень дохода на 4 категории по уровням: низкий, средний, выше среднего и высокий

In [ ]:
quantile_1 = credit_score_data['total_income'].quantile([0.25]).sum()
quantile_2 = credit_score_data['total_income'].quantile([0.5]).sum() 
quantile_3 = credit_score_data['total_income'].quantile([0.75]).sum()

In [ ]:
#определим функцию возвращающую уровень дохода
def categorize_income(value):
    if value <= quantile_1 :
        return 'Низкий'
    elif quantile_1  < value <= quantile_2 :
        return 'Средний'
    elif quantile_2 < value <= quantile_3:
        return 'Выше среднего'
    elif  value > quantile_3:
        return 'Высокий'
    
#добаввим в таблицу столбец с новыми категориями по уровню дохода
credit_score_data['lvl_income'] = credit_score_data['total_income'].apply(categorize_income)
credit_score_data

**Вывод**

Мы преобразовали наши данные, обогатив их распределение на категории (по целям кредитования) и уровню дохода.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
child_group_count = credit_score_data.groupby('children').count()['debt'].sort_index()
child_group_debt = credit_score_data.groupby('children').sum()['debt'].sort_index()
print('Доля проблемных кредиторов в группе в зависимости от количества детей', (child_group_debt / child_group_count) * 100, sep='\n')
print()
print('Количество кредиторов в группе по количеству детей', child_group_count, sep='\n')

In [ ]:
credit_score_data.groupby('children')[['debt']].agg(['count', 'sum', 'mean']).style.format({('debt', 'mean'): '{:.2%}'})

**Вывод**

Из представленных наблюдений можно сделать несколько выводов:
С количеством детей количество кредиторов уменьшается, т.е. люди начинают реже брать кредиты. Скорее всего это связано с тем, что дети появляются с возрастом, и соответственно основные затраты (свадьба, дом и автомобиль) были понесены до появления детей (возможно влияет выплата материнского капитала).
Также, в зависимости от группы (по кол-ву детей) не наблюдается сильное различие в доли должников, она варьируется от 8% до 10%. Можно сказать, что в целом у банка 10% должников - проблемные.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_status_group = credit_score_data.groupby('family_status')['debt'].count().sort_index()
family_status_group_count = credit_score_data.groupby('family_status')['debt'].sum().sort_index()
print('Доля проблемных должников в зависимости от семейного положения \n', ((family_status_group_count / family_status_group) * 100).sort_values().round())

**Вывод**

Можно наблюдать некоторую зависимость: среди людей не состоящих в браке доля на 3% выше, чем разведенных (3% от тысяч клиентов банка - довольно существенная доля). Скорее всего это связано с тем, что развод обычно наступает в достаточно зрелом возрасте, и в обществе тенденция такая, что с возрастом доходы растут, соответственно люди успешнее выплачивают кредит.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
income_group_count = credit_score_data.groupby('lvl_income').count()['debt'].sort_index()
income_group_sum = credit_score_data.groupby('lvl_income').sum()['debt'].sort_index()
 
print(((income_group_sum / income_group_count) * 100).round())

In [ ]:
credit_score_data.groupby('lvl_income')['debt'].sum().sort_values()

**Вывод**

Доля проблемных кредиторов с высоким доходом ниже, чем в остальных группах (7% "плохих" должников в группе с высоким уровнем дохода, и 9% в остальных группах): более обеспеченные люди успешнее погашают займ, в то время как клиенты с меньшем уровнем дохода чаще имеют пробемы с погашением задолженности.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_group_count = credit_score_data.groupby('new_purp_category').count()['debt']
purpose_group_sum = credit_score_data.groupby('new_purp_category').sum()['debt']
 
print(((purpose_group_sum / purpose_group_count) * 100).round())
print()
print(purpose_group_count)

In [ ]:
credit_score_data.groupby('new_purp_category')['debt'].sum().sort_values()

**Вывод**

Исходя из наших наблюдений можно сказать что проблемы с погашением задолженности по кредиту чаще встречаются у клиентов, котоыре приобретают автомобиль или недвижимость, это закономерно: стоимость недвижимости обычно выше, чем суммы по другим категориям, что влечет за собой наибольшуюж финансовую нагрузку. 

Свадьба имеет наименьшее количество проблемных должников, т.к. с погашением кредита на нее могут помочь родственники, или же молодожёны могут погасить часть кредита из собранных средств. 

Довольно высокое число проблемных дебиторов, взявших кредит на образование может объясняться тем, что зачастую это могут быть неработающие студенты, или люди с невысоким заработком, которым необходимо образование, чтобы зарабатывать больше.е.

В целом, доля должников по группам различается и варьируется в диапазоне от 8% до 10%.   Можно сказать, что цель кредитования оказывает влияние на уровень доли проблемных должников, но не является главным фактором.

## Шаг 4. Общий вывод

В ходе анализа базы данных, мы обнаружили проблемы в сборе или хранении данных, которые приводят к их искажению, что в дальнейшем может приводить которые к удалению значений в столбцах дохода и опыта работы.

Мы обнаружили 54 дубликата, что вероятно говорит о повторном внесении данных о кредиторе, либо повторном взятии кредита по одной и той же цели. В любом случае необходим дополнительный анализ дубликатов уже с идентификатором заемщика.

Было обнаружено, что наиболее наиболее проблемные кредиторы - это люди с большим количеством детей и низкой зарплатой.
И риск неплатежеспособности кредитора растет с увеличением количества детей и/или уменишением уровня заработной платы.
Также мы выяснили, что наименьший риск для банка - это выдача кредита на свадьбу. 

Более объемные и репрезентативные данные могут помочь сделать более точный анализ.